This tutorial will show how to create and load objects representing legal Holdings in AuthoritySpoke.

To get ready, we need to repeat some setup steps we already saw in the `Introduction to AuthoritySpoke` guide. First, import the package.

In [1]:
import authorityspoke
from authorityspoke.io.downloads import download_case
from authorityspoke.io.loaders import load_decision

Again, you have the choice of using either the real API clients or mockups that supply only the testing data for these examples.

In [2]:
USE_REAL_CASE_API = False
USE_REAL_LEGISLICE_API = False

Next, we can download the judicial decisions we're going to compare.

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

if USE_REAL_CASE_API:
    CAP_API_KEY = os.getenv('CAP_API_KEY')
    
    oracle_download = download_case(
    cite="750 F.3d 1339", 
    full_case=True, 
    api_key=CAP_API_KEY)
    
    lotus_download = download_case(
    cite="49 F.3d 807", 
    full_case=True, 
    api_key=CAP_API_KEY)

else:
    oracle_download = load_decision("oracle_h.json")
    lotus_download = load_decision("lotus_h.json")

Then we convert the JSON responses from the API into AuthoritySpoke `Opinion` objects.

In [4]:
from authorityspoke.io.readers import read_decision

oracle = read_decision(oracle_download).majority
lotus = read_decision(lotus_download).majority

And we need a `Client` for accessing legislative provisions.

In [5]:
import json

from authorityspoke.io.downloads import Client, FakeClient

if USE_REAL_LEGISLICE_API:
    
    LEGISLICE_API_TOKEN = os.getenv("LEGISLICE_API_TOKEN")
    legis_client = Client(api_token=LEGISLICE_API_TOKEN)

else:
    legis_client = FakeClient.from_file("usc.json")

## Loading Holdings from Existing JSON

Now we're ready to look at the process of describing legal `Holding`s and loading that information into AuthoritySpoke. In version 0.4, although there's not yet a web interface for loading this data, there is an interface for loading JSON files, and there's an OpenAPI schema specification for the input data (see below). 

Although there are interfaces for loading Authorityspoke objects in the `authorityspoke.io.loaders` and `authorityspoke.io.schemas` modules, the most useful way to load data is to create a JSON document that contains a list of objects, where each object represents one Holding representing a list of Holdings. Then you can load the Holdings into AuthoritySpoke objects using the `loaders.load_and_read_holdings` function.

In [6]:
from authorityspoke.io.loaders import load_and_read_holdings

oracle_holdings = load_and_read_holdings("holding_oracle.json", client=legis_client)
lotus_holdings = load_and_read_holdings("holding_lotus.json", client=legis_client)

If you want to open one of the input JSON files in your own text editor for comparison, you can find them in the folder `example_data/holdings/`.

`holding_oracle.json` contains a list of holdings. These are places where the text of the _Oracle_ opinion endorses legal rules (or sometimes, rejects legal rules). Each of these rules is described procedurally, in terms of inputs and outputs. 

Each holding in the JSON input may also include an `anchors` field indicating where the holding can be found in the opinion. For instance, the first holding of _Oracle America v. Google_ is derived from the following sentence from the majority opinion:

> By statute, a work must be “original” to qualify for copyright protection. 17 U.S.C. § 102(a).

The `anchors` field doesn't do much yet in AuthoritySpoke version 0.4, but in future versions it'll help link each Holding to the relevant parts of the Opinion.

## The Parts of a Holding in JSON

Now let's look at the part of `holding_oracle.json` representing that first holding.

In [7]:
from authorityspoke.io.loaders import load_holdings

holdings_to_read = load_holdings("holding_oracle.json")
holdings_to_read[0]

{'inputs': {'type': 'fact',
  'content': '{the Java API} was an original work',
  'truth': False,
  'anchors': 'a work must be “original”'},
 'outputs': {'type': 'fact',
  'content': 'the Java API was copyrightable',
  'truth': False,
  'anchors': 'must be “original” to qualify for |copyright protection.|'},
 'mandatory': True,
 'enactments': {'node': '/us/usc/t17/s102/a',
  'exact': 'Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device.',
  'name': 'copyright protection provision',
  'anchors': 'qualify for copyright protection. |17 U.S.C. § 102(a)|.'},
 'anchors': 'By statute, a work |must be “original” to qualify| for'}

To compare the input data to the created Python objects, you can link the Holdings to the Opinions using the `.posit` method. As we look at the parts of the JSON file, the code cells will show how fields from the JSON affect the structure of the Holding object.

In [8]:
oracle.posit(oracle_holdings)
lotus.posit(lotus_holdings)

print(oracle.holdings[0])

the Holding to ACCEPT
  the Rule that the court MUST SOMETIMES impose the
    RESULT:
      the Fact it is false that <the Java API> was copyrightable
    GIVEN:
      the Fact it is false that <the Java API> was an original work
    GIVEN the ENACTMENT:
      "Copyright protection subsists, in accordance with this title, in original works of authorship fixed in any tangible medium of expression, now known or later developed, from which they can be perceived, reproduced, or otherwise communicated, either directly or with the aid of a machine or device.…" (/us/usc/t17/s102/a 2013-07-18)


This Holding means that according to the cited enactment, if it's false that "the Java API was an original work", then it's mandatory for the court to find it to be false that "the Java API was copyrightable".

The JSON file represented these Factors inside an "inputs" field (labeled as the "GIVEN" Factors when you print the Holding object) and an "outputs" field (labeled as "RESULT" Factors). Inputs are the preconditions for applying the Holding, and outputs are the results. Not shown here, Rules can also have "despite" Factors, which are Factors that don't need to be present to trigger the rule, but that don't prevent the rule from applying if they're present. There can be more than one Factor in the "inputs", "outputs" or "despite" categories, and if so they would be listed together in square brackets in the JSON.

In [9]:
print(oracle.holdings[0].inputs[0])

the Fact it is false that <the Java API> was an original work


The curly brackets around `{the Java API}` indicate that the parser should consider that phrase to be a reference to an Entity object, which becomes one of the input's `context_factors`. If such an object hasn't been referenced before in the file, it will be created.

In [10]:
print(oracle.holdings[0].inputs[0].context_factors)

(Entity(name='the Java API', generic=True, plural=False, anchors=[]),)


The JSON representation of a Rule can also have "mandatory" and "universal" fields. If omitted, the values of these fields are implied as False. "universal" means that the Rule applies whenever its inputs are present. "mandatory" means that when Rule applies, the court has no discretion and must accept the outputs.

In [11]:
print(oracle.holdings[0].mandatory)

True


The JSON can also contain fields representing Enactments. It identifies a passage of legislative text with a [United States Legislative Markup](https://github.com/usgpo/uslm) identifier that shows the "path" to the text. In this case, "us" refers to the jurisdiction (the US federal government), "usc" refers to the Code (the United States Code), "t17" specifies Title 17 of the United States Code, "s102" specifies Section 102 of Title 17, and "a" specifies subsection (a) of Section 102. If the relevant passage is less than the entire section or subsection, an "exact" field can identify the full text of the passage or "prefix" and "suffix" fields can be used to the phrase by what comes immediately before or after it. You don't need to include "prefix" and "suffix" if you're sure the phrase you're trying to select only occurs once in the statute subdivision you've cited. Alternatively, a passage can be saved as a `text` field with pipe characters that split it into three parts for "prefix", "exact", and "suffix" fields.

For instance, to get just the phrase "original works of authorship", we could have included the following field in the JSON when loading:
```
"text": "in accordance with this title, in|original works of authorship|fixed"
```

We can also select that same string to change the Enactment's selected text after loading the Enactment:

In [12]:
to_select = "in accordance with this title, in|original works of authorship|fixed"
oracle.holdings[0].enactments[0].select(to_select)

And we can use the `selected_text` method to verify that the Enactment's selected text has changed.

In [13]:
oracle.holdings[0].enactments[0].selected_text()

'…original works of authorship…'

The "name" field is a nickname that can be used to refer to the passage again later in the same file. For any Factor or Enactment object, you can add a "name" field and assign a unique string value as the name. If you need to refer to the object again in the list of Holdings you're importing, you can replace the object with the name string. This means a Holding object could have "input", "despite" and "output" fields containing lists of string indentifiers of Factors defined elsewhere. Enactment objects can be replaced the same way in the "enactments" and "enactments_despite" fields.

In [14]:
holdings_to_read[0]["enactments"]["name"]

'copyright protection provision'

In the second holding in the JSON file, you can see where the enactment is referenced by its name "copy protection provision" instead of being repeated in its entirety.

In [15]:
holdings_to_read[1]

{'inputs': [{'type': 'fact',
   'content': 'the Java API was independently created by the author, as opposed to copied from other works',
   'anchors': 'the work was independently created by the author (as opposed to copied from other works)'},
  {'type': 'fact',
   'content': 'the Java API possessed at least some minimal degree of creativity',
   'anchors': 'it possesses at least some minimal degree of creativity.'}],
 'outputs': {'type': 'fact',
  'content': 'the Java API was an original work',
  'anchors': 'Original, as the term is used in copyright'},
 'mandatory': True,
 'universal': True,
 'enactments': 'copyright protection provision'}

There can also be an "enactments_despite" field, which identifies legislative text that doesn't need to be present for the Rule to apply, but that also doesn't negate the validity of the Rule.

## JSON API Specification

If you want to view the schema specification, you can find it in the `io.api_spec` module. When you read it, you might be surprised to see that every Holding object contains a Rule, and every Rule contains a Procedure. 

If you prefer, instead of nesting a Rule object and Procedure object inside the Holding object, AuthoritySpoke's data loading library allows you to place all the properties of the Rule and the Procedure directly into the Holding object, as shown in the examples above.

In [16]:
from authorityspoke.io.api_spec import make_spec

yaml = make_spec().to_yaml()
print(yaml)

components:
  schemas:
    Allegation:
      properties:
        absent:
          default: false
          type: boolean
        anchors:
          items:
            $ref: '#/components/schemas/Selector'
          type: array
        generic:
          default: false
          type: boolean
        name:
          default: null
          nullable: true
          type: string
        pleading:
          allOf:
          - $ref: '#/components/schemas/Pleading'
          default: null
          nullable: true
        statement:
          allOf:
          - $ref: '#/components/schemas/Fact'
          default: null
          nullable: true
      type: object
    CrossReference:
      properties:
        reference_text:
          type: string
        target_node:
          type: integer
        target_uri:
          type: string
        target_url:
          format: url
          type: string
      required:
      - reference_text
      - target_uri
      - target_url
      type: object
  

## Exporting AuthoritySpoke Holdings back to JSON

Finally, if you want to convert an AuthoritySpoke object back to JSON or to a Python dictionary, you can do so with the `io.dump` module. Although no API exists yet for serving and ingesting data using the AuthoritySpoke Holding Schema, this JSON format is easier to store and share over the web.

In [17]:
from authorityspoke.io import dump

dump.to_json(oracle.holdings[0].outputs[0])

'{"predicate": {"content": "{} was copyrightable", "reciprocal": false, "quantity": null, "truth": false, "comparison": ""}, "context_factors": [{"generic": true, "name": "the Java API", "plural": false, "anchors": [], "type": "Entity"}], "generic": false, "name": "false the Java API was copyrightable", "anchors": [{"exact": "copyright protection.", "prefix": "must be \\u201coriginal\\u201d to qualify for ", "suffix": ""}, {"exact": "whether the non-literal elements of a program \\u201care protected", "prefix": "", "suffix": ""}], "absent": false, "standard_of_proof": null}'